In [ ]:
import eli5
import matplotlib.pyplot as plt
import pandas as pd

from pathlib import Path
from XPER.viz.Visualisation import visualizationClass as viz

from src.explanation.performance.xper import xper_values
from src.explanation.performance.permutation_importance import pi
from src.modeling.create_data_split import split_data
from src.utils.models_pkl import load_pickle

from config.config_modeling import CAT_COLS, RANDOM_STATE

# Initializing all the necessary variables

In [ ]:
OUT_PATH = Path("../../data") / "data.csv"
MODEL_PATH = Path("../../models") / "XGB.pkl"

In [ ]:
df = pd.read_csv(OUT_PATH)

In [ ]:
model = load_pickle(MODEL_PATH)

In [ ]:
data = split_data(cols=CAT_COLS, df=df, random_state=RANDOM_STATE)

In [ ]:
X_train, y_train, X_test, y_test = (
    data["train"][0],
    data["train"][1],
    data["test"][0],
    data["test"][1],
)

# XPER Values

In [ ]:
# Get the perfomance metric as well as the XPER values for the model's performance for train data
PM_test, XPER_values = xper_values(X_train, y_train, X_test, y_test, model)

In [ ]:
# Print the performance metrics
print("Performance Metrics: ", PM_test)

In [ ]:
labels = list(X_train.columns)

In [ ]:
# bar plot to measure contribution of each feature to the AUC
viz.bar_plot(
    XPER_values=XPER_values, X_test=pd.DataFrame(X_test), labels=labels, p=38, percentage=True
)

In [ ]:
viz.beeswarn_plot(XPER_values=XPER_values, X_test=pd.DataFrame(X_test), labels=labels)

In [ ]:
viz.force_plot(
    XPER_values=XPER_values, instance=1, X_test=X_test, variable_name=labels, figsize=(16, 4)
)

# Permutation Importance

In [ ]:
sorted_features, sorted_contributions, perm_importance = pi(X_train, X_test, y_test, model)

# Display feature importances using weights
weights = eli5.show_weights(perm_importance, feature_names=list(X_train.columns))

In [ ]:
# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_features)), sorted_contributions, tick_label=sorted_features)
plt.title("Percentage Contribution to AUC (Permutation Importance)")
plt.xlabel("Features")
plt.ylabel("Percentage Contribution (%)")
plt.xticks(rotation=90)
plt.show()